# 리포트 작성 에이전트

일주일 정도 쌓인 일기 데이터를 분석하여 리포트를 작성하는 에이전트

In [1]:
# 필요한 라이브러리 import
from datetime import datetime, timedelta
from typing import TypedDict, Optional, List
from langgraph.graph import StateGraph, END
import os
import json
from dotenv import load_dotenv
from langchain_upstage import ChatUpstage

load_dotenv()
api_key = os.getenv("UPSTAGE_API_KEY")

# LLM 초기화
chat = ChatUpstage(
    model="solar-pro2-251215",
    upstage_api_key=api_key
)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### State 정의

class ReportGenerationState(TypedDict):
    """리포트 생성 에이전트의 State"""
    diary_entries: List[dict]  # extractor로 분석된 일기 항목들 [{"date": "...", "content": "...", "topic": "...", "emotion": "..."}]
    period_start: str  # 리포트 기간 시작일 "YYYY-MM-DD"
    period_end: str  # 리포트 기간 종료일 "YYYY-MM-DD"
    report: Optional[str]  # 생성된 리포트 내용
    summary: Optional[str]  # 리포트 요약

In [3]:
### 노드 함수 정의

def analyze_diary_data(state: ReportGenerationState) -> ReportGenerationState:
    """일기 데이터 분석 노드 - 일주일치 데이터를 분석"""
    diary_entries = state["diary_entries"]
    period_start = state["period_start"]
    period_end = state["period_end"]
    
    print(f"[analyze_diary_data] 일기 데이터 분석 중...")
    print(f"  - 리포트 기간: {period_start} ~ {period_end}")
    print(f"  - 일기 항목 수: {len(diary_entries)}개")
    
    # 감정별 통계
    emotions = [entry.get("emotion", "") for entry in diary_entries if entry.get("emotion")]
    emotion_counts = {}
    for emotion in emotions:
        emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1
    
    # 주제별 통계
    topics = [entry.get("topic", "") for entry in diary_entries if entry.get("topic")]
    topic_counts = {}
    for topic in topics:
        topic_counts[topic] = topic_counts.get(topic, 0) + 1
    
    print(f"  - 감정 분포: {emotion_counts}")
    print(f"  - 주요 주제: {dict(sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)[:5])}")
    
    return state

def generate_report(state: ReportGenerationState) -> ReportGenerationState:
    """리포트 생성 노드 - LLM을 사용하여 리포트 작성"""
    diary_entries = state["diary_entries"]
    period_start = state["period_start"]
    period_end = state["period_end"]
    
    # 일기 데이터를 날짜순으로 정렬
    sorted_entries = sorted(diary_entries, key=lambda x: x.get("date", ""))
    
    # 일기 데이터 요약 (최대 20개)
    entries_summary = "\n".join([
        f"- {entry.get('date', '')}: [{entry.get('topic', 'N/A')}] [{entry.get('emotion', 'N/A')}] {entry.get('content', '')[:50]}"
        for entry in sorted_entries[-20:]  # 최근 20개
    ]) if sorted_entries else "일기 데이터 없음"
    
    # 감정 통계
    emotions = [entry.get("emotion", "") for entry in diary_entries if entry.get("emotion")]
    emotion_counts = {}
    for emotion in emotions:
        emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1
    
    # 주제 통계
    topics = [entry.get("topic", "") for entry in diary_entries if entry.get("topic")]
    topic_counts = {}
    for topic in topics:
        topic_counts[topic] = topic_counts.get(topic, 0) + 1
    
    # 주요 주제 상위 5개
    top_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)[:5]
    
    # LLM 호출을 위한 프롬프트 구성
    prompt = f"""일주일간의 일기 데이터를 분석하여 리포트를 작성하세요.

=== 리포트 기간 ===
시작일: {period_start}
종료일: {period_end}
일기 항목 수: {len(diary_entries)}개

=== 일기 데이터 ===
{entries_summary}

=== 통계 정보 ===
감정 분포: {emotion_counts}
주요 주제 (상위 5개): {top_topics}

=== 리포트 작성 지침 ===
1. 이번 주 동안의 주요 감정 변화와 패턴을 분석
2. 자주 나타난 주제와 그 의미를 설명
3. 전반적인 감정 상태와 웰빙에 대한 통찰 제공
4. 간결하고 읽기 쉬운 형식으로 작성
5. 개인적인 톤으로 작성 (너무 딱딱하지 않게)

JSON 형식:
{{
  "report": "리포트 전체 내용 (3-5 문단)",
  "summary": "한 문장 요약"
}}"""
    
    # LLM 호출
    response = chat.invoke(prompt)
    
    # JSON 파싱
    try:
        # 응답에서 JSON 추출
        response_text = response.content.strip()
        
        # JSON 부분만 추출
        if "```json" in response_text:
            json_start = response_text.find("```json") + 7
            json_end = response_text.find("```", json_start)
            response_text = response_text[json_start:json_end].strip()
        elif "```" in response_text:
            json_start = response_text.find("```") + 3
            json_end = response_text.find("```", json_start)
            response_text = response_text[json_start:json_end].strip()
        
        # 첫 번째 JSON 객체만 추출
        if "{" in response_text:
            start_idx = response_text.find("{")
            brace_count = 0
            end_idx = start_idx
            
            for i in range(start_idx, len(response_text)):
                if response_text[i] == "{":
                    brace_count += 1
                elif response_text[i] == "}":
                    brace_count -= 1
                    if brace_count == 0:
                        end_idx = i + 1
                        break
            
            response_text = response_text[start_idx:end_idx]
        
        result_json = json.loads(response_text)
        state["report"] = result_json.get("report", "")
        state["summary"] = result_json.get("summary", "")
        
        print(f"[generate_report] 리포트 생성 완료")
        print(f"  - 리포트 요약: {state['summary']}")
        
    except Exception as e:
        print(f"[generate_report] JSON 파싱 실패: {e}")
        print(f"[generate_report] 원본 응답: {response.content[:200]}...")
        # 기본값 사용
        state["report"] = "리포트 생성 실패"
        state["summary"] = "리포트 생성에 실패했습니다"
    
    return state

In [4]:
### 그래프 구성

# StateGraph 생성
workflow = StateGraph(ReportGenerationState)

# 노드 추가
workflow.add_node("analyze_diary_data", analyze_diary_data)
workflow.add_node("generate_report", generate_report)

# 엣지 추가 - 단순 선형 흐름
workflow.set_entry_point("analyze_diary_data")
workflow.add_edge("analyze_diary_data", "generate_report")
workflow.add_edge("generate_report", END)

# 그래프 컴파일
app = workflow.compile()

print("✅ 리포트 작성 에이전트가 생성되었습니다!")

✅ 리포트 작성 에이전트가 생성되었습니다!


In [5]:
### 리포트 생성 함수 (간편 사용을 위한 래퍼 함수)

def generate_weekly_report(diary_entries: List[dict], period_start: str = None, period_end: str = None) -> dict:
    """
    일주일치 일기 데이터로 리포트를 생성하는 간편 함수
    
    Args:
        diary_entries: extractor로 분석된 일기 항목 리스트
        period_start: 리포트 기간 시작일 (선택, 기본값: 일주일 전)
        period_end: 리포트 기간 종료일 (선택, 기본값: 오늘)
    
    Returns:
        리포트 딕셔너리 (report, summary 포함)
    """
    if period_end is None:
        period_end = datetime.now().strftime("%Y-%m-%d")
    if period_start is None:
        period_start = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")
    
    initial_state = ReportGenerationState(
        diary_entries=diary_entries,
        period_start=period_start,
        period_end=period_end,
        report=None,
        summary=None
    )
    
    result = app.invoke(initial_state)
    
    return {
        "report": result.get("report", ""),
        "summary": result.get("summary", ""),
        "period_start": result.get("period_start", period_start),
        "period_end": result.get("period_end", period_end)
    }

In [6]:
### 테스트 실행

# 테스트 데이터 - 일주일치 일기 항목 (extractor로 분석된 결과)
test_diary_entries = [
    {"date": "2026-01-08", "content": "아 부장 ㅅㅂ 화나네 회의때깨짐", "topic": "부장회의", "emotion": "빡침"},
    {"date": "2026-01-09", "content": "야식 먹어서 살찌겟네 ㅠ", "topic": "야식", "emotion": "후회"},
    {"date": "2026-01-10", "content": "내일 친구 5시 고깃집", "topic": "친구 약속", "emotion": "기대"},
    {"date": "2026-01-11", "content": "내일 회의 있음", "topic": "회의", "emotion": "중립"},
    {"date": "2026-01-12", "content": "오늘 하루가 힘들었어요", "topic": "일상", "emotion": "슬픔"},
    {"date": "2026-01-13", "content": "프로젝트 발표 준비 완료", "topic": "프로젝트 발표", "emotion": "긴장"},
    {"date": "2026-01-14", "content": "오늘은 프로젝트 발표 준비를 했다", "topic": "프로젝트 발표", "emotion": "긴장"},
]

# 리포트 생성
result = generate_weekly_report(
    diary_entries=test_diary_entries,
    period_start="2026-01-08",
    period_end="2026-01-14"
)

print("\n=== 리포트 결과 ===")
print(f"\n기간: {result['period_start']} ~ {result['period_end']}")
print(f"\n요약: {result['summary']}")
print(f"\n리포트:\n{result['report']}")

[analyze_diary_data] 일기 데이터 분석 중...
  - 리포트 기간: 2026-01-08 ~ 2026-01-14
  - 일기 항목 수: 7개
  - 감정 분포: {'빡침': 1, '후회': 1, '기대': 1, '중립': 1, '슬픔': 1, '긴장': 2}
  - 주요 주제: {'프로젝트 발표': 2, '부장회의': 1, '야식': 1, '친구 약속': 1, '회의': 1}
[generate_report] 리포트 생성 완료
  - 리포트 요약: 이번 주는 프로젝트 발표 준비와 업무 스트레스로 긴장감과 부정적 감정이 우세했으나, 친구 만남 등 작은 낙으로 균형을 유지한 주간이었다.

=== 리포트 결과 ===

기간: 2026-01-08 ~ 2026-01-14

요약: 이번 주는 프로젝트 발표 준비와 업무 스트레스로 긴장감과 부정적 감정이 우세했으나, 친구 만남 등 작은 낙으로 균형을 유지한 주간이었다.

리포트:
이번 주는 업무 스트레스와 프로젝트 준비로 인한 긴장감이 주를 이뤘어요. 초반에는 부장님과의 불편한 회의로 '빡침'이 쌓였고, 야식 후회로 작은 우울감이 있었지만, 친구와의 만남을 앞두고 '기대감'으로 전환됐어요. 중반부에는 '중립'적인 회의와 '슬픔'이 교차하며 에너지가 떨어진 모습이 보였는데, 주말 직전부터는 프로젝트 발표 준비로 '긴장' 상태가 2회나 기록되며 집중 모드로 전환된 게 특징이에요.

가장 빈번한 주제는 '프로젝트 발표'였는데, 이는 업무 압박의 핵심 원인으로 보여요. '부장회의'와 '회의' 기록은 상사와의 관계나 업무 환경에서 스트레스가 지속되고 있음을 시사해요. 반면 '친구 약속'은 일상의 작은 낙으로 작용했고, '야식'은 자기관리의 약점을 드러내는 계기가 됐어요.

종합적으로 보면, 업무 관련 부정적 감정(빡침, 긴장, 슬픔)이 4건으로 절반 이상을 차지해 웰빙 관리가 필요해 보여요. 주말에도 프로젝트 준비에 매몰된 점에서 휴식 부족이 우려되니, 다음 주에는 작은 성취라도 축하하는 시간을 가져보는 게 좋을 것 